In [1]:
from dotenv import load_dotenv
import os
import uuid
import chromadb
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from chromadb.utils import embedding_functions
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from kiwipiepy import Kiwi
import gradio as gr
import time
import csv

In [2]:
from dotenv import load_dotenv
import os

# .env 파일 활성화
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("jyl_langgraph_test") #set_enable=False

LangSmith 추적을 시작합니다.
[프로젝트명]
jyl_langgraph_test


# Documents

## database 연결하고 collection 생성

In [1]:
## 컬렉션 생성/연결하기
import chromadb
client = chromadb.PersistentClient('chroma/')
client.list_collections()

[Collection(name=csv_files_openai_3072), Collection(name=49_files_openai_3072)]

In [5]:
collection = client.get_or_create_collection(name="49_files_openai_3072")
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [10]:
# client.delete_collection(name="12_files_openai_3072")

In [11]:
# collection.modify(name="12_files_openai_3072") # Rename the collection

## 데이터 불러오기

In [6]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader("[1018] 주소정보_업무편람_최종(하이퍼링크).docx")

data = loader.load()

data

[Document(page_content='새로운 주소체계를 도입하여 정착·발전시키는 중차대한 임무를 훌륭히 수행해온 주소정보 업무 담당 공무원 여러분! \n\n도로명주소가 국민 생활 속에 안착되고 있는 것은 여러분께서 도로명주소를 부여하고 안내시설을 설치하기 위해 현장을 직접 발로 뛴 노고의 결과입니다.\n\n여러분께서 실시간 갱신하는 주소정보는 통신·택배 등 생활편의 분야 뿐만 아니라 긴급구조·방범 등 안전분야에서도 위치 찾기를 빠르고 정확하게 획기적으로 발전시켜 왔습니다.\n\n한편, 지금까지의 평면도로와 건물중심의 주소체계로는 입체복합공간이나 사물 등의 위치 표시에 어려움이 있고, 특히 4차 산업혁명 시대의 도래와 예기치 않은 코로나19로 인한 배달 등 비대면 문화의 확산은 모든 공간과 접점에 주소가 필요하다는 요구로 이어졌습니다.\n\n이에 따라 ‘21년 6월 9일 도로명주소법을 전부 개정하여 입체주소·사물주소 도입 등 주소체계를 고도화하고, 주소정보 관련 국민의 신청권을 확대하는 한편, 자율주행, 음성인식 등 신기술의 등장에 대응하여 주소정보 기반의 새로운 산업을 지원하는 근거를 마련하였습니다.\n\n이번 업무편람에는 법령 및 행정규칙의 제·개정 사항을 모두 반영하였으며, 법령의 내용만으로 이해가 어려운 경우를 대비하여 구체적인 업무사례를 제시하고자 노력하였습니다. 지방자치단체에서 주소정보 업무를 수행하는 데 도움이 되기를 기대합니다.\n\n여러분이 만드는 주소정보는 국민의 생활편의와 안전을 지키는 국가의 자산이자 현실 세계와 디지털 세계를 연결하는 매개체로써 미래사회 혁신성장을 선도하는 기초가 될 것입니다. 주소정보를 다루는 공무원 여러분을 응원합니다. 고맙습니다.\n\n2021.  9.\n\n주소정책 가족 일동\n\n본 업무편람은 「도로명주소법」에 따른 주소제도의 발자취, 주소정보의 개념, 도로명주소의 부여·변경·폐지, 도로명주소대장의 작성·관리, 주소정보시설의 설치 및 유지관리, 사물주소 부여 등 선진화 제도 등과 관련한 절차와 처리기준, 처리

In [3]:
import olefile
ole = olefile.OleFileIO('[1018] 주소정보_업무편람_최종(하이퍼링크).hwp')
encoded_text = ole.openstream('PrvText').read()
decoded_text = encoded_text.decode('UTF-16')

decoded_text

' \r\n\r\n<주소정보 업무편람을 발간하면서>\r\n\r\n새로운 주소체계를 도입하여 정착·발전시키는 중차대한 임무를 훌륭히 수행해온 주소정보 업무 담당 공무원 여러분! \r\n도로명주소가 국민 생활 속에 안착되고 있는 것은 여러분께서 도로명주소를 부여하고 안내시설을 설치하기 위해 현장을 직접 발로 뛴 노고의 결과입니다.\r\n여러분께서 실시간 갱신하는 주소정보는 통신·택배 등 생활편의 분야 뿐만 아니라 긴급구조·방범 등 안전분야에서도 위치 찾기를 빠르고 정확하게 획기적으로 발전시켜 왔습니다.\r\n한편, 지금까지의 평면도로와 건물중심의 주소체계로는 입체복합공간이나 사물 등의 위치 표시에 어려움이 있고, 특히 4차 산업혁명 시대의 도래와 예기치 않은 코로나19로 인한 배달 등 비대면 문화의 확산은 모든 공간과 접점에 주소가 필요하다는 요구로 이어졌습니다.\r\n이에 따라 ‘21년 6월 9일 도로명주소법을 전부 개정하여 입체주소·사물주소 도입 등 주소체계를 고도화하고, 주소정보 관련 국민의 신청권을 확대하는 한편, 자율주행, 음성인식 등 신기술의 등장에 대응하여 주소정보 기반의 새로운 산업을 지원하는 근거를 마련하였습니다.\r\n이번 업무편람에는 법령 및 행정규칙의 제·개정 사항을 모두 반영하였으며, 법령의 내용만으로 이해가 어려운 경우를 대비하여 구체적인 업무사례를 제시하고자 노력하였습니다. 지방자치단체에서 주소정보 업무를 수행하는 데 도움이 되기를 기대합니다.\r\n여러분이 만드는 주소정보는 국민의 생활편의와 안전을 지키는 국가의 자산이자 현실 세계와 디지털 세계를 연결하는 매개체로써 미래사회 혁신성장을 선도하는 기초가 될 것입니다. 주소정보를 다루는 공무원 여러분을 응원합니다. 고맙습니다.\r\n2021.  9.\r\n주소정책 가족 일동\r\n<[행정안전부] 지방자치분권실장 박성호, 지역발전정책관 김명선, 주소정책과 김창남, 박종각, 최방남, 고종신, 이인아, 박재호, 윤미정, 최인량, 엄경철, 김도현, 신성심, 신준오, 이민희 [지방자치단체] 강정훈

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('data/final/도로명주소법.txt')
data = loader.load()

print(type(data))
data

<class 'list'>


[Document(metadata={'source': 'data/final/도로명주소법.txt'}, page_content='도로명주소법\n[시행 2021. 6. 9.] [법률 제17574호, 2020. 12. 8., 전부개정]\n행정안전부(주소생활공간과) 044-205-3567\n\n제1조(목적) 이 법은 도로명주소, 국가기초구역, 국가지점번호 및 사물주소의 표기ㆍ사용ㆍ관리ㆍ활용 등에 관한 사항을 규정함으로써 국민의 생활안전과 편의를 도모하고 관련 산업의 지원을 통하여 국가경쟁력 강화에 이바지함을 목적으로 한다.\n\n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.\n1. “도로”란 다음 각 목의 어느 하나에 해당하는 것을 말한다.\n가. 「도로법」 제2조제1호에 따른 도로(같은 조 제2호에 따른 도로의 부속물은 제외한다)\n나. 그 밖에 차량 등 이동수단이나 사람이 통행할 수 있는 통로로서 대통령령으로 정하는 것\n2. “도로구간”이란 도로명을 부여하기 위하여 설정하는 도로의 시작지점과 끝지점 사이를 말한다.\n3. “도로명”이란 도로구간마다 부여된 이름을 말한다.\n4. “기초번호”란 도로구간에 행정안전부령으로 정하는 간격마다 부여된 번호를 말한다.\n5. “건물번호”란 다음 각 목의 어느 하나에 해당하는 건축물 또는 구조물(이하 “건물등”이라 한다)마다 부여된 번호(둘 이상의 건물등이 하나의 집단을 형성하고 있는 경우로서 대통령령으로 정하는 경우에는 그 건물등의 전체에 부여된 번호를 말한다)를 말한다.\n가. 「건축법」 제2조제1항제2호에 따른 건축물\n나. 현실적으로 30일 이상 거주하거나 정착하여 활동하는 데 이용되는 인공구조물 및 자연적으로 형성된 구조물\n6. “상세주소”란 건물등 내부의 독립된 거주ㆍ활동 구역을 구분하기 위하여 부여된 동(棟)번호, 층수 또는 호(號)수를 말한다.\n7. “도로명주소”란 도로명, 건물번호 및 상세주소(상세주소가 있는 경우만 해당한다)로 표기하는 주소를 말한다.\n8. “국가기초구역”이란

In [9]:
loader = DirectoryLoader(path='data/final', glob='주소체계 고도화에 따른 주소정보 개념 정립 연구.pdf', loader_cls=PyMuPDFLoader, show_progress=True, use_multithreading=True)
data = loader.load()
len(data)

100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


246

In [10]:
loader = DirectoryLoader(path='data/final', glob='주소정보_업무편람_최종(하이퍼링크).pdf', loader_cls=PyMuPDFLoader, show_progress=True, use_multithreading=True)
data = loader.load()
len(data)

100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


525

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, CSVLoader, UnstructuredMarkdownLoader, UnstructuredWordDocumentLoader
import os

# Custom loader class that selects the appropriate loader based on file extension
class MultiLoader:
    def __init__(self, path):
        self.path = path

    def load(self):
        return self._get_loader().load()

    def lazy_load(self):
        return self._get_loader().lazy_load()

    def _get_loader(self):
        loaders = {
            '.pdf': PyMuPDFLoader,
            '.txt': TextLoader,
            '.md': UnstructuredMarkdownLoader,
            '.docx': UnstructuredWordDocumentLoader,
            # Add more extensions and their corresponding loaders if needed
        }
        ext = os.path.splitext(self.path)[1].lower()
        loader_cls = loaders.get(ext, None)
        if loader_cls is None:
            raise ValueError(f"No loader found for file extension: {ext}")
        return loader_cls(self.path)

# DirectoryLoader with the custom loader class
loader = DirectoryLoader(path='data/final', glob='*', loader_cls=MultiLoader, show_progress=True, use_multithreading=True)
data = loader.load()

100%|██████████| 49/49 [00:06<00:00,  7.15it/s]


In [7]:
data

[Document(page_content='도로명주소법\n[시행 2021. 6. 9.] [법률 제17574호, 2020. 12. 8., 전부개정]\n행정안전부(주소생활공간과) 044-205-3567\n\n제1조(목적) 이 법은 도로명주소, 국가기초구역, 국가지점번호 및 사물주소의 표기ㆍ사용ㆍ관리ㆍ활용 등에 관한 사항을 규정함으로써 국민의 생활안전과 편의를 도모하고 관련 산업의 지원을 통하여 국가경쟁력 강화에 이바지함을 목적으로 한다.\n\n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.\n1. “도로”란 다음 각 목의 어느 하나에 해당하는 것을 말한다.\n가. 「도로법」 제2조제1호에 따른 도로(같은 조 제2호에 따른 도로의 부속물은 제외한다)\n나. 그 밖에 차량 등 이동수단이나 사람이 통행할 수 있는 통로로서 대통령령으로 정하는 것\n2. “도로구간”이란 도로명을 부여하기 위하여 설정하는 도로의 시작지점과 끝지점 사이를 말한다.\n3. “도로명”이란 도로구간마다 부여된 이름을 말한다.\n4. “기초번호”란 도로구간에 행정안전부령으로 정하는 간격마다 부여된 번호를 말한다.\n5. “건물번호”란 다음 각 목의 어느 하나에 해당하는 건축물 또는 구조물(이하 “건물등”이라 한다)마다 부여된 번호(둘 이상의 건물등이 하나의 집단을 형성하고 있는 경우로서 대통령령으로 정하는 경우에는 그 건물등의 전체에 부여된 번호를 말한다)를 말한다.\n가. 「건축법」 제2조제1항제2호에 따른 건축물\n나. 현실적으로 30일 이상 거주하거나 정착하여 활동하는 데 이용되는 인공구조물 및 자연적으로 형성된 구조물\n6. “상세주소”란 건물등 내부의 독립된 거주ㆍ활동 구역을 구분하기 위하여 부여된 동(棟)번호, 층수 또는 호(號)수를 말한다.\n7. “도로명주소”란 도로명, 건물번호 및 상세주소(상세주소가 있는 경우만 해당한다)로 표기하는 주소를 말한다.\n8. “국가기초구역”이란 도로명주소를 기반으로 국토를 읍ㆍ면ㆍ동의 면적보다 작게 경계를 정하여 나눈 구역을 말한다.\n9

In [8]:
len(data)

294

In [ ]:
# ## 한번에 여러 파일 불러오기
# from langchain_community.document_loaders import DirectoryLoader
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.document_loaders import TextLoader
# loader = DirectoryLoader(path='data', glob='*.txt', loader_cls= TextLoader, show_progress=True, use_multithreading=True)
# data = loader.load()
# data

## Collection에 저장하기 (재실행X)

In [9]:
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions

def insert(content, collection_name):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = OPENAI_API_KEY,
                model_name="text-embedding-3-large" ## text-embedding-ada-002
            )
    collection = client.get_collection(name=collection_name, embedding_function=openai_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content, metadatas = doc.metadata)

In [10]:
insert(data, '49_files_openai_3072')

100%|██████████| 3189/3189 [27:17<00:00,  1.95it/s] 


In [8]:
insert(data, 'tmp')

100%|██████████| 2783/2783 [27:53<00:00,  1.66it/s]  


100%|██████████| 4466/4466 [42:56<00:00,  1.73it/s]  


In [ ]:
## Child Docs 생성
# * BM25를 위한 Docs
# * Collection에 저장했을 때와 똑같은 chunk_size, overlap_size로 설정하기

# import uuid
# import chromadb
# import uuid
# from tqdm import tqdm
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# id_key = "doc_id"
# doc_ids = [str(uuid.uuid4()) for _ in data]
# child_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)

# child_docs = []  # 하위 문서를 저장할 리스트를 초기화합니다.
# for i, doc in enumerate(data):
#     _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
#     # 현재 문서를 하위 문서로 분할합니다.
#     child_doc = child_text_splitter.split_documents([doc])
#     for _doc in child_doc:  # 분할된 하위 문서에 대해 반복합니다.
#         # 하위 문서의 메타데이터에 ID를 저장합니다.
#         _doc.metadata[id_key] = _id
#     child_docs.extend(child_doc)  # 분할된 하위 문서를 리스트에 추가합니다.

# print(len(child_docs))
# child_docs

# CSV

## database 연결하고 collection 생성

In [18]:
## 컬렉션 생성/연결하기
import chromadb
client = chromadb.PersistentClient('chroma/')
client.list_collections()

[Collection(name=12_files_openai_3072), Collection(name=csv_files_openai_3072)]

In [27]:
collection = client.get_or_create_collection(name="csv_files_openai_3072")
collection.peek()

{'ids': ['8b2d008c-4417-11ef-bfb1-ae08445dcffd',
  '8b95e728-4417-11ef-bfb1-ae08445dcffd',
  '8c07a19c-4417-11ef-bfb1-ae08445dcffd',
  '8c54f17c-4417-11ef-bfb1-ae08445dcffd',
  '8ca8f6fa-4417-11ef-bfb1-ae08445dcffd',
  '8d06aa20-4417-11ef-bfb1-ae08445dcffd',
  '8e5e5670-4417-11ef-bfb1-ae08445dcffd',
  '8eb7f69e-4417-11ef-bfb1-ae08445dcffd',
  '8f4916ba-4417-11ef-bfb1-ae08445dcffd',
  '8f964e8a-4417-11ef-bfb1-ae08445dcffd'],
 'embeddings': [[0.021085666492581367,
   -0.0005870306049473584,
   -0.0142073268070817,
   0.019306784495711327,
   0.007708482909947634,
   0.012985829263925552,
   0.0002616436977405101,
   -0.009736406616866589,
   -0.03790201619267464,
   -0.04679642245173454,
   0.004473885055631399,
   -0.026730647310614586,
   -0.016318265348672867,
   -0.01138483639806509,
   0.026493463665246964,
   -0.0005677594454027712,
   -0.03555389493703842,
   0.017883680760860443,
   0.01846478134393692,
   0.01895100809633732,
   -0.006872409023344517,
   0.021239835768938065,
  

In [ ]:
# client.delete_collection(name="ten_files_openai_3072")

## 데이터 불러오기

In [34]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, CSVLoader, UnstructuredMarkdownLoader
import os

# Custom loader class that selects the appropriate loader based on file extension
class MultiLoader:
    def __init__(self, path):
        self.path = path

    def load(self):
        return self._get_loader().load()

    def lazy_load(self):
        return self._get_loader().lazy_load()

    def _get_loader(self):
        loaders = {
            # '.pdf': PyMuPDFLoader,
            # '.txt': TextLoader,
            '.csv': CSVLoader,
            # '.md': UnstructuredMarkdownLoader,
            # Add more extensions and their corresponding loaders if needed
        }
        ext = os.path.splitext(self.path)[1].lower()
        loader_cls = loaders.get(ext, None)
        if loader_cls is None:
            raise ValueError(f"No loader found for file extension: {ext}")
        return loader_cls(self.path)

# DirectoryLoader with the custom loader class
loader = DirectoryLoader(path='data/csv', glob='*.csv', loader_cls=MultiLoader, show_progress=True, use_multithreading=True)
data = loader.load()

100%|██████████| 1/1 [00:00<00:00, 82.37it/s]


In [36]:
data

[Document(metadata={'source': 'data/csv/주소용어최종정리_1123 - 업데이트_231123.csv', 'row': 0}, page_content='번호: 89\n필수여부: 필수\n준용여부: 미준용\n용어명: 1~2차선\n정의: 도로에서 차량이 이동할 수 있는 영역으로, 한 번에 최대 2대의 차량이 나란히 이동할 수 있는 구간\n선정근거_출처: 업무편람\n선정근거_세부내용: p365\n주소지식모델 포함여부: 포함\n주소지식모델정의: 도로에서 차량이 이동할 수 있는 영역으로, 한 번에 최대 2대의 차량이 나란히 이동할 수 있는 구간\n유사용어: '),
 Document(metadata={'source': 'data/csv/주소용어최종정리_1123 - 업데이트_231123.csv', 'row': 1}, page_content='번호: 90\n필수여부: 필수\n준용여부: 미준용\n용어명: 100m국가지점번호격자\n정의: 100m 크기의 정사각형 격자\n선정근거_출처: 업무편람\n선정근거_세부내용: \n주소지식모델 포함여부: 포함\n주소지식모델정의: 격자참조체계를 따르는 국가지점번호참조체계의 주소부여대상 단위 중의 하나로 국토조사에 사용되는 격자체계의 단위 중 한변의 길이가 100m인 정사각형인 격자\n유사용어: '),
 Document(metadata={'source': 'data/csv/주소용어최종정리_1123 - 업데이트_231123.csv', 'row': 2}, page_content='번호: 91\n필수여부: 필수\n준용여부: 미준용\n용어명: 10km국가지점번호격자\n정의: 10km 크기의 정사각형 격자\n선정근거_출처: 업무편람\n선정근거_세부내용: \n주소지식모델 포함여부: 포함\n주소지식모델정의: 격자참조체계를 따르는 국가지점번호참조체계의 주소부여대상 단위 중의 하나로 국토조사에 사용되는 격자체계의 단위 중 한변의 길이가 10km인 정사각형인 격자\n유사용어: '),
 Document(metadata={'source': '

In [37]:
len(data)

509

In [ ]:
# ## 한번에 여러 파일 불러오기
# from langchain_community.document_loaders import DirectoryLoader
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.document_loaders import TextLoader
# loader = DirectoryLoader(path='data', glob='*.txt', loader_cls= TextLoader, show_progress=True, use_multithreading=True)
# data = loader.load()
# data

## Collection에 저장하기 (재실행X)

In [40]:
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions

def insert(content, collection_name):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = OPENAI_API_KEY,
                model_name="text-embedding-3-large" ## text-embedding-ada-002
            )
    collection = client.get_collection(name=collection_name, embedding_function=openai_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content, metadatas = doc.metadata)

In [41]:
insert(data, 'csv_files_openai_3072')

100%|██████████| 509/509 [05:10<00:00,  1.64it/s]


In [42]:
collection.count()

558

In [29]:
collection.peek()

{'ids': ['8b2d008c-4417-11ef-bfb1-ae08445dcffd',
  '8b95e728-4417-11ef-bfb1-ae08445dcffd',
  '8c07a19c-4417-11ef-bfb1-ae08445dcffd',
  '8c54f17c-4417-11ef-bfb1-ae08445dcffd',
  '8ca8f6fa-4417-11ef-bfb1-ae08445dcffd',
  '8d06aa20-4417-11ef-bfb1-ae08445dcffd',
  '8e5e5670-4417-11ef-bfb1-ae08445dcffd',
  '8eb7f69e-4417-11ef-bfb1-ae08445dcffd',
  '8f4916ba-4417-11ef-bfb1-ae08445dcffd',
  '8f964e8a-4417-11ef-bfb1-ae08445dcffd'],
 'embeddings': [[0.021085666492581367,
   -0.0005870306049473584,
   -0.0142073268070817,
   0.019306784495711327,
   0.007708482909947634,
   0.012985829263925552,
   0.0002616436977405101,
   -0.009736406616866589,
   -0.03790201619267464,
   -0.04679642245173454,
   0.004473885055631399,
   -0.026730647310614586,
   -0.016318265348672867,
   -0.01138483639806509,
   0.026493463665246964,
   -0.0005677594454027712,
   -0.03555389493703842,
   0.017883680760860443,
   0.01846478134393692,
   0.01895100809633732,
   -0.006872409023344517,
   0.021239835768938065,
  

In [ ]:
## Child Docs 생성
# * BM25를 위한 Docs
# * Collection에 저장했을 때와 똑같은 chunk_size, overlap_size로 설정하기

# import uuid
# import chromadb
# import uuid
# from tqdm import tqdm
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# id_key = "doc_id"
# doc_ids = [str(uuid.uuid4()) for _ in data]
# child_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)

# child_docs = []  # 하위 문서를 저장할 리스트를 초기화합니다.
# for i, doc in enumerate(data):
#     _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
#     # 현재 문서를 하위 문서로 분할합니다.
#     child_doc = child_text_splitter.split_documents([doc])
#     for _doc in child_doc:  # 분할된 하위 문서에 대해 반복합니다.
#         # 하위 문서의 메타데이터에 ID를 저장합니다.
#         _doc.metadata[id_key] = _id
#     child_docs.extend(child_doc)  # 분할된 하위 문서를 리스트에 추가합니다.

# print(len(child_docs))
# child_docs